In [ ]:
# conversion between 2 llms about joke

# imports

import os
from dotenv import load_dotenv
from openai import OpenAI, AzureOpenAI
import anthropic
from IPython.display import Markdown, display, update_display


import base64
# from openai import AzureOpenAI


In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
# anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
# google_api_key = os.getenv('GOOGLE_API_KEY')


# personalise env data FOR AZURE OPENAI
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT_URL")
azure_openai_deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o-mini")
azure_openai_subscription_key = os.getenv("AZURE_OPENAI_API_KEY")


# personalise env data FOR DEEPSEEK AZURE OPENAI
azure_deepseek_endpoint = os.getenv("DEEPSEEK_AZURE_INFERENCE_SDK_ENDPOINT")
azure_deepseek_deployment= os.getenv("DEPLOYMENT_NAME", "DeepSeek-V3-0324")
azure_deepseek_subscription_key = os.getenv("DEEPSEEK_AZURE_INFERENCE_SDK_KEY")

# personalise env data FOR GROK AZURE OPENAI
azure_grok_endpoint = os.getenv("GROK_AZURE_INFERENCE_SDK_ENDPOINT")
azure_grok_deployment = os.getenv("GROK_DEPLOYMENT_NAME", "grok-3-mini")
azure_grok_subscription_key = os.getenv("GROK_AZURE_INFERENCE_SDK_KEY")


# anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
# google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OPENAI API Key not set")
if azure_openai_subscription_key:
    print(f"AZURE OpenAI API Key exists and begins {azure_openai_subscription_key[:8]}")
else:
    print("AZURE OPENAI API Key not set")
    
# if anthropic_api_key:
#     print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
# else:
#     print("Anthropic API Key not set")

# if google_api_key:
#     print(f"Google API Key exists and begins {google_api_key[:8]}")
# else:
#     print("Google API Key not set")
if azure_deepseek_subscription_key:
    print(f"DEEPSEEK AZURE API Key exists and begins {azure_deepseek_subscription_key[:7]}")
else:
    print("DEEPSEEK AZURE API Key not set")

if azure_grok_subscription_key:
    print(f"GROK_AZURE API Key exists and begins {azure_grok_subscription_key[:8]}")
else:
    print("GROK_AZURE API Key not set")

In [ ]:
# Connect to OpenAI
openai = OpenAI()


In [ ]:
system_message = "You are an assistant that is great at telling jokes \
a short summary, ignoring text that might be navigation related. \
try as much as possible not to exceed 1000 words. \
Respond in markdown."
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [ ]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [ ]:
# GPT AZURE OPENAIS
# Initialize Azure OpenAI client with key-based authentication
clientGPTAzure = AzureOpenAI(
    azure_endpoint=azure_openai_endpoint,
    api_key=azure_openai_subscription_key,
    api_version="2025-01-01-preview",
)

In [ ]:

# Include speech result if speech is enabled
messages = prompts

# Generate the completion
completion = clientGPTAzure.chat.completions.create(
    model=azure_openai_deployment,
    messages=messages,
    max_tokens=6553,
    temperature=0.7,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=False
)

# print(completionto_json())
print(completion.choices[0].message.content)

In [ ]:
# DEEPSEEK AZURE OPENAI
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
# from dotenv import load_dotenv

clientDeepSeekAzure = ChatCompletionsClient(endpoint=azure_deepseek_endpoint, credential=AzureKeyCredential(azure_deepseek_subscription_key))

In [ ]:
sysContent="You are an assistant that is great at telling jokes \
a short summary, ignoring text that might be navigation related. \
try as much as possible not to exceed 1000 words. \
Respond in markdown."
usrContent="Tell a light-hearted joke for an audience of Data Scientists, response should be in markdown format"


In [ ]:

response = clientDeepSeekAzure.complete(
  messages=[
    SystemMessage(content=sysContent),
    UserMessage(content=usrContent)
  ],
  model = azure_deepseek_deployment,
  max_tokens=1000
)

# print(response)
print(response.choices[0].message.content)


In [ ]:
# GROK AZURE OPENAI

clientGrokAzure = ChatCompletionsClient(endpoint=azure_grok_endpoint, credential=AzureKeyCredential(azure_grok_subscription_key))

In [ ]:

response = clientGrokAzure.complete(
  messages=[
    SystemMessage(content=sysContent),
    UserMessage(content=usrContent)
  ],
  model = azure_grok_deployment,
  max_tokens=1000
)

# print(response)
print(response.choices[0].message.content)

In [ ]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [ ]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4.1-mini',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

In [ ]:
# Let's make a conversation between GPT-4.1-mini and Claude-3.5-haiku
# We're using cheap versions of models so the costs will be minimal


azure_openai_gpt_model = "gpt-4o-mini"
azure_deepseek_model = azure_deepseek_deployment

azure_openai_gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

azure_deepseek_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."


azure_openai_gpt_messages = ["Hi there"]
azure_deepseek_messages = ["Hi"]

In [ ]:
def call_gpt():
    # print(f"inside call_gpt,Before zip function \n azure_openai_gpt_messages: \n {azure_openai_gpt_messages} \n azure_deepseek_messages \n{azure_deepseek_messages} ")
    messages = [{"role": "system", "content": azure_openai_gpt_system}]
    for gpt, deepseek in zip(azure_openai_gpt_messages, azure_deepseek_messages):
        # print(f"picking each value from gpt, deepseek \n gpt i/p message: \n {gpt} \n deepseek i/p message \n{deepseek} ")
    
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": deepseek})
    # completion = openai.chat.completions.create(
    #     model=gpt_model,
    #     messages=messages
    # )
    # return completion.choices[0].message.content

    # Include speech result if speech is enabled
    # messages = prompts

    # Generate the completion
    completion = clientGPTAzure.chat.completions.create(
    model=azure_openai_deployment,
    messages=messages,
    max_tokens=6553,
    temperature=0.7,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=False
    )

    # print(completionto_json())
    return (completion.choices[0].message.content)

In [ ]:
call_gpt()

In [ ]:

def call_azure_deepseek():
    # print(f"inside call_azure_deepseek,Before zip function \n azure_openai_gpt_messages: \n {azure_openai_gpt_messages} \n azure_deepseek_messages \n{azure_deepseek_messages} ")
    
    messages = []
    for gpt, azure_deepseek_message in zip(azure_openai_gpt_messages, azure_deepseek_messages):
        # print(f"picking each value from gpt, azure_deepseek_message \n gpt i/p message: \n {gpt} \n azure_deepseek_message i/p message \n{azure_deepseek_message} ")
    
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": azure_deepseek_message})
        messages.append({"role": "user", "content": azure_openai_gpt_messages[-1]})
    # message = claude.messages.create(
    #     model=claude_model,
    #     system=claude_system,
    #     messages=messages,
    #     max_tokens=500
    # )
    message = clientDeepSeekAzure.complete(
        messages=messages,
      model=azure_deepseek_deployment,
      max_tokens=1000
    )

    # print(response)
    # print(response.choices[0].message.content)
    return message.choices[0].message.content

In [ ]:
 call_azure_deepseek()

In [ ]:
call_gpt()

In [ ]:

azure_openai_gpt_messages = ["Hi there"]
azure_deepseek_messages = ["Hi"]

print(f"Azure OpenAI GPT:\n{azure_openai_gpt_messages[0]}\n")
print(f"Azure Deepseek:\n{azure_deepseek_messages[0]}\n")

for i in range(2):
    gpt_next = call_gpt()
    print(f"Azure OpenAI GPT:\n{gpt_next}\n")
    azure_openai_gpt_messages.append(gpt_next)
    
    azure_deepseek_next = call_azure_deepseek()
    print(f"Azure Deepseek:\n{azure_deepseek_next}\n")
    azure_deepseek_messages.append(azure_deepseek_next)